In [2]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [2]:
#부산데이터로 모델 학습시키기
df=pd.read_csv('부산광역시데이터셋_최종.csv', encoding='euckr')
df.head()

,index_부산 보행등,관리번호,구,동,지번,교차로,고가,구분,재질,종류,...,스마트 음향신호기,음향신호기,전체신호등,주민센터,지하철,터미널,편의점,약국,병원,관할구별 시각장애인 수
0,1,9-945-07,해운대구,석대동,538-2,NaN,지상,보행등,LED,1면2색,...,64.176430,470.833404,64.176430,1022.863654,314.344945,4231.693661,145.445152,814.169768,557.720095,1724
1,2,9-945-02,해운대구,석대동,514-3,NaN,지상,보행등,LED,1면2색,...,60.387322,487.416004,60.387322,1042.188031,306.005236,4246.960340,133.241979,811.377867,575.154499,1724
2,3,9-943-07,해운대구,우동,977-8,NaN,지상,보행등,LED,1면2색,...,93.621086,18.921870,18.921870,211.149550,499.571548,1680.802262,285.997005,19.001587,19.001587,1724
3,4,9-943-03,해운대구,우동,977-8,NaN,지상,보행등,LED,1면2색,...,106.553277,31.532652,31.532652,199.002832,490.075206,1673.688035,274.359834,31.584851,31.584851,1724
4,5,9-942-12,해운대구,반송동,62-605,NaN,지상,보행등,LED,1면2색,...,22.034601,67.417662,22.034601,383.085229,355.408493,7248.261659,150.637497,50.977776,39.082138,1724


In [3]:
df.columns

Index(['index_부산 보행등', '관리번호', '구', '동', '지번', '교차로', '고가', '구분', '재질', '종류',
       '신호등종류', '경도', '위도', '교육시설', '도서관', '버스정류소', '복지관', '시각장애인협회', '안마원',
       '스마트 음향신호기', '음향신호기', '전체신호등', '주민센터', '지하철', '터미널', '편의점', '약국', '병원',
       '관할구별 시각장애인 수'],
      dtype='object')

In [4]:
df['label'] = np.where(df['스마트 음향신호기'] < 150, 1, 0)
df=df.drop(['스마트 음향신호기'],axis=1, inplace=False)
df=df.drop(['관리번호', '구', '동', '지번', '교차로', '고가', '구분', '재질', '종류',
       '신호등종류', '경도', '위도'],axis=1, inplace=False)
df

,index_부산 보행등,교육시설,도서관,버스정류소,복지관,시각장애인협회,안마원,음향신호기,전체신호등,주민센터,지하철,터미널,편의점,약국,병원,관할구별 시각장애인 수,label
0,1,2417.879656,12705.060760,141.605483,1037.862825,12413.490090,1701.868620,470.833404,64.176430,1022.863654,314.344945,4231.693661,145.445152,814.169768,557.720095,1724,1
1,2,2435.553629,12720.660220,126.183552,1051.315521,12433.327920,1713.321603,487.416004,60.387322,1042.188031,306.005236,4246.960340,133.241979,811.377867,575.154499,1724,1
2,3,6444.897700,14024.588570,75.101932,1137.888200,9737.092056,777.585976,18.921870,18.921870,211.149550,499.571548,1680.802262,285.997005,19.001587,19.001587,1724,1
3,4,6458.035113,14035.173160,68.832494,1127.685462,9741.790396,787.771837,31.532652,31.532652,199.002832,490.075206,1673.688035,274.359834,31.584851,31.584851,1724,1
4,5,5933.995947,16237.446680,36.608591,39.897317,15465.223390,186.414343,67.417662,22.034601,383.085229,355.408493,7248.261659,150.637497,50.977776,39.082138,1724,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9049,9050,13164.708470,9764.163274,59.501073,1150.794805,2943.170470,631.621888,42.195113,42.195113,284.405242,250.403414,711.740290,85.875810,88.657403,68.252058,312,0
9050,9051,13136.090140,9782.988604,82.203006,1146.496242,2923.297271,629.519271,63.694237,63.694237,310.892781,301.131537,685.109598,115.136919,118.531701,77.334476,312,0
9051,9052,13123.891020,9764.341811,65.867680,1129.115339,2908.745138,611.774408,48.693005,48.693005,292.922164,302.739372,672.012179,124.810210,128.102822,59.278163,312,0
9052,9053,13335.198000,9818.774662,59.812683,1087.443708,3100.839539,765.410954,216.371851,47.509757,371.050538,77.510091,884.281205,38.117834,47.473434,47.473434,312,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9054 entries, 0 to 9053
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index_부산 보행등  9054 non-null   int64  
 1   교육시설          9054 non-null   float64
 2   도서관           9054 non-null   float64
 3   버스정류소         9054 non-null   float64
 4   복지관           9054 non-null   float64
 5   시각장애인협회       9054 non-null   float64
 6   안마원           9054 non-null   float64
 7   음향신호기         9054 non-null   float64
 8   전체신호등         9054 non-null   float64
 9   주민센터          9054 non-null   float64
 10  지하철           9054 non-null   float64
 11  터미널           9054 non-null   float64
 12  편의점           9054 non-null   float64
 13  약국            9054 non-null   float64
 14  병원            9054 non-null   float64
 15  관할구별 시각장애인 수  9054 non-null   int64  
 16  label         9054 non-null   int32  
dtypes: float64(14), int32(1), int64(2)
memory usage: 1.1 MB


In [6]:
df.describe()

,index_부산 보행등,교육시설,도서관,버스정류소,복지관,시각장애인협회,안마원,음향신호기,전체신호등,주민센터,지하철,터미널,편의점,약국,병원,관할구별 시각장애인 수,label
count,9054.000000,9054.000000,9054.000000,9054.000000,9054.000000,9054.000000,9054.000000,9054.000000,9054.000000,9054.000000,9054.000000,9054.000000,9054.000000,9054.000000,9054.000000,9054.000000,9054.000000
mean,4527.500000,11211.907405,11509.415767,83.759791,1323.865769,10897.769491,1757.669133,978.154944,648.375118,1088.835351,2701.601313,5433.161866,261.576222,525.484083,410.444004,1177.103601,0.374199
std,2613.809002,7024.763654,5721.438637,96.452867,1440.257534,6414.604919,2283.931190,1453.170206,1310.836765,1401.960419,3720.177405,4075.837667,359.417496,838.150820,667.540241,479.050224,0.483942
min,1.000000,94.914708,132.135043,0.423618,12.498616,103.211486,8.937651,3.989834,3.989834,10.403904,3.590091,41.780512,4.061726,5.888781,6.203348,312.000000,0.000000
25%,2264.250000,5933.995484,7752.086682,30.515187,446.170881,6289.812516,431.568968,198.937176,57.613116,299.840183,415.247968,2352.384159,79.358027,101.087153,65.877966,806.000000,0.000000
50%,4527.500000,9403.183679,11023.640800,54.736090,821.384148,9489.014310,795.230620,434.199497,174.669215,516.962936,830.754773,4183.971714,148.473166,227.549926,166.416951,1238.000000,0.000000
75%,6790.750000,15615.429458,13883.030527,101.506264,1471.486273,14208.504297,1931.305378,1044.953095,492.474297,1030.986287,2999.408454,7529.284639,273.286939,460.626275,386.952533,1724.000000,1.000000
max,9054.000000,32452.566760,32600.883940,1444.369021,10565.250900,33716.078580,12145.648850,9551.322988,9349.971509,6861.291763,17018.411820,21239.371510,4255.561713,7459.996088,5513.734041,1831.000000,1.000000


In [7]:
df.dtypes

index_부산 보행등      int64
교육시설            float64
도서관             float64
버스정류소           float64
복지관             float64
시각장애인협회         float64
안마원             float64
음향신호기           float64
전체신호등           float64
주민센터            float64
지하철             float64
터미널             float64
편의점             float64
약국              float64
병원              float64
관할구별 시각장애인 수      int64
label             int32
dtype: object

In [8]:
df=df.drop(['전체신호등'],axis=1, inplace=False)

In [9]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df,df['label'], test_size=0.2, random_state=42, \
                                                    stratify=df['label'])

x_train=x_train.drop(['index_부산 보행등','label'],axis=1)
x_test=x_test.drop(['index_부산 보행등','label'],axis=1)

In [10]:
from sklearn.preprocessing import RobustScaler

scaler=['교육시설', '도서관', '버스정류소', '복지관', '시각장애인협회', '안마원', '음향신호기', '주민센터', '지하철', '터미널', '편의점', '약국', '병원','관할구별 시각장애인 수']
rbs = RobustScaler()
x_train[scaler]=rbs.fit_transform(x_train[scaler])
x_test[scaler]=rbs.fit_transform(x_test[scaler])

In [11]:
from sklearn.preprocessing import RobustScaler

scaler=['교육시설', '도서관', '버스정류소', '복지관', '시각장애인협회', '안마원', '음향신호기', '주민센터', '지하철', '터미널', '편의점', '약국', '병원','관할구별 시각장애인 수']
rbs = RobustScaler()
df[scaler]=rbs.fit_transform(df[scaler])

In [12]:
#서울데이터로 모델돌리고 결과 저장

df2=pd.read_csv('서울데이터셋_최종.csv', encoding='euckr')
df2.head()

,신호등_index,관리번호,관할구,주소,X좌표,Y좌표,교육시설,도서관,버스정류소,복지관,시각장애인협회,안마원,음향신호기,주민센터,지하철,터미널,편의점,약국,병원,관할구별 시각장애인 수
0,1,03-0000013905,강남구,강남구 대치동 973-12 대,205374.9878,544360.3235,8828.504490,1272.720157,63.539989,1346.410759,2169.162096,465.230504,1.183158,284.085577,522.826511,4951.938755,38.526782,41.464028,37.449544,1551
1,2,03-0000064343,강남구,강남구 역삼동 712 공,203974.3625,544362.8187,7609.817483,2564.026031,56.791601,889.563384,3466.957845,1500.918235,1.322839,346.584163,724.241328,3578.020909,164.459202,109.535357,84.957371,1551
2,3,03-0000065097,강남구,강남구 역삼동 633-16 대,202722.1824,544404.9315,6557.443582,3724.820316,88.930535,605.690941,4686.685859,1433.985553,3.073867,469.181845,293.056979,2362.775142,168.592329,26.863093,26.863093,1551
3,4,03-0000014413,강남구,강남구 삼성동 186-6천,205839.8146,546182.1334,8603.064701,2586.538263,110.674797,2188.520623,3196.645816,1718.649139,330.012641,325.610619,2239.093625,3299.653565,93.347071,263.335148,247.677596,1551
4,5,03-0000066954,강남구,강남구 일원동 306유,207106.7500,544477.2562,10352.677580,1048.237676,396.732442,921.576718,1160.556333,891.721938,397.458902,792.651712,644.402218,4058.568839,442.528962,541.561810,599.914988,1551


In [13]:
df.corr()

,index_부산 보행등,교육시설,도서관,버스정류소,복지관,시각장애인협회,안마원,음향신호기,주민센터,지하철,터미널,편의점,약국,병원,관할구별 시각장애인 수,label
index_부산 보행등,1.000000,0.521482,0.008324,0.054392,0.287597,0.147181,0.328358,0.229878,0.338150,0.279766,0.344835,0.237009,0.261547,0.249477,-0.450335,-0.134995
교육시설,0.521482,1.000000,0.351734,0.177468,0.646405,0.438889,0.704852,0.561819,0.737025,0.747769,0.783145,0.381254,0.538096,0.548819,-0.509429,-0.229807
도서관,0.008324,0.351734,1.000000,0.155850,0.403211,0.787443,0.401409,0.515428,0.413348,0.651778,0.556749,0.179700,0.311617,0.373284,-0.279042,-0.164922
버스정류소,0.054392,0.177468,0.155850,1.000000,0.236680,0.211825,0.262415,0.298425,0.181547,0.227048,0.175303,0.259776,0.291150,0.342392,-0.109575,-0.158333
복지관,0.287597,0.646405,0.403211,0.236680,1.000000,0.563049,0.902488,0.707141,0.673940,0.786841,0.688880,0.540575,0.703948,0.691005,-0.478927,-0.294228
시각장애인협회,0.147181,0.438889,0.787443,0.211825,0.563049,1.000000,0.600511,0.628555,0.575608,0.776473,0.709166,0.305091,0.476168,0.502014,-0.334977,-0.286133
안마원,0.328358,0.704852,0.401409,0.262415,0.902488,0.600511,1.000000,0.776661,0.763296,0.836734,0.765558,0.553678,0.751699,0.740055,-0.524410,-0.317689
음향신호기,0.229878,0.561819,0.515428,0.298425,0.707141,0.628555,0.776661,1.000000,0.570571,0.764796,0.665189,0.463592,0.736863,0.745400,-0.409497,-0.318005
주민센터,0.338150,0.737025,0.413348,0.181547,0.673940,0.575608,0.763296,0.570571,1.000000,0.798136,0.792819,0.377384,0.591929,0.651834,-0.535965,-0.281713
지하철,0.279766,0.747769,0.651778,0.227048,0.786841,0.776473,0.836734,0.764796,0.798136,1.000000,0.855362,0.419100,0.642851,0.671472,-0.593463,-0.320982


In [14]:
df2.columns

Index(['신호등_index', '관리번호', '관할구', '주소', 'X좌표', 'Y좌표', '교육시설 ', '도서관', '버스정류소',
       '복지관', '시각장애인협회', '안마원', '음향신호기', '주민센터', '지하철 ', '터미널 ', '편의점 ', '약국',
       '병원', '관할구별 시각장애인 수'],
      dtype='object')

In [15]:
df2=df2.drop(['관리번호', '관할구', '주소', 'X좌표', 'Y좌표'],axis=1, inplace=False)
df2

,신호등_index,교육시설,도서관,버스정류소,복지관,시각장애인협회,안마원,음향신호기,주민센터,지하철,터미널,편의점,약국,병원,관할구별 시각장애인 수
0,1,8828.504490,1272.720157,63.539989,1346.410759,2169.162096,465.230504,1.183158,284.085577,522.826511,4951.938755,38.526782,41.464028,37.449544,1551
1,2,7609.817483,2564.026031,56.791601,889.563384,3466.957845,1500.918235,1.322839,346.584163,724.241328,3578.020909,164.459202,109.535357,84.957371,1551
2,3,6557.443582,3724.820316,88.930535,605.690941,4686.685859,1433.985553,3.073867,469.181845,293.056979,2362.775142,168.592329,26.863093,26.863093,1551
3,4,8603.064701,2586.538263,110.674797,2188.520623,3196.645816,1718.649139,330.012641,325.610619,2239.093625,3299.653565,93.347071,263.335148,247.677596,1551
4,5,10352.677580,1048.237676,396.732442,921.576718,1160.556333,891.721938,397.458902,792.651712,644.402218,4058.568839,442.528962,541.561810,599.914988,1551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24068,24069,7230.450246,2681.460032,35.737890,298.156171,1917.693375,1624.475557,2.641973,47.062448,2701.702998,1160.385186,84.847481,33.221670,57.142015,2094
24069,24070,7230.450246,2681.460032,35.737890,298.156171,1917.693375,1624.475557,2.641973,47.062448,2701.702998,1160.385186,84.847481,33.221670,57.142015,2094
24070,24071,8283.776312,4607.601742,0.736851,1820.796008,3763.578659,1073.949851,8.947163,514.306317,4036.730398,956.527579,170.793254,154.857993,106.395229,2094
24071,24072,8282.427777,4510.148534,25.926075,1685.624452,3672.525345,1079.266733,142.404324,357.276709,3993.819568,865.386136,63.388223,9.139335,25.484132,2094


In [16]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24073 entries, 0 to 24072
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   신호등_index     24073 non-null  int64  
 1   교육시설          24073 non-null  float64
 2   도서관           24073 non-null  float64
 3   버스정류소         24073 non-null  float64
 4   복지관           24073 non-null  float64
 5   시각장애인협회       24073 non-null  float64
 6   안마원           24073 non-null  float64
 7   음향신호기         24073 non-null  float64
 8   주민센터          24073 non-null  float64
 9   지하철           24073 non-null  float64
 10  터미널           24073 non-null  float64
 11  편의점           24073 non-null  float64
 12  약국            24073 non-null  float64
 13  병원            24073 non-null  float64
 14  관할구별 시각장애인 수  24073 non-null  int64  
dtypes: float64(13), int64(2)
memory usage: 2.8 MB


In [17]:
from sklearn.preprocessing import RobustScaler

scaler=['도서관', '버스정류소','복지관', '시각장애인협회', '안마원', '음향신호기', '주민센터', '지하철 ', '터미널 ', '편의점 ', '약국','병원', '관할구별 시각장애인 수']
rbs = RobustScaler()
df2[scaler] = rbs.fit_transform(df2[scaler])

#df2=df2.drop(['신호등_index'],axis=1)

In [18]:
df2.corr()

,신호등_index,교육시설,도서관,버스정류소,복지관,시각장애인협회,안마원,음향신호기,주민센터,지하철,터미널,편의점,약국,병원,관할구별 시각장애인 수
신호등_index,1.000000,-0.294719,0.247429,0.006248,0.092895,-0.350726,-0.015612,-0.057390,-0.017147,-0.075061,0.160397,-0.010704,-0.009836,-0.013320,-0.316008
교육시설,-0.294719,1.000000,-0.049331,0.138891,-0.001126,0.678580,0.404095,0.140441,0.235233,0.145676,-0.100733,0.149245,0.141762,0.112230,0.378783
도서관,0.247429,-0.049331,1.000000,-0.029465,0.172125,0.017484,0.135759,0.102645,0.076421,-0.173473,0.324794,0.045010,0.050923,0.041502,-0.232975
버스정류소,0.006248,0.138891,-0.029465,1.000000,0.021792,0.082278,0.110362,0.179871,0.129950,0.021911,-0.063323,0.200209,0.223889,0.225581,0.051554
복지관,0.092895,-0.001126,0.172125,0.021792,1.000000,0.103153,0.218512,0.132442,0.229134,0.200857,0.088604,0.222557,0.178214,0.163816,-0.107973
시각장애인협회,-0.350726,0.678580,0.017484,0.082278,0.103153,1.000000,0.344639,0.183156,0.234665,0.404220,-0.095437,0.121264,0.140277,0.120135,0.296553
안마원,-0.015612,0.404095,0.135759,0.110362,0.218512,0.344639,1.000000,0.162532,0.368348,0.190681,0.015218,0.320678,0.296666,0.275840,0.075021
음향신호기,-0.057390,0.140441,0.102645,0.179871,0.132442,0.183156,0.162532,1.000000,0.255012,0.040482,-0.010585,0.180607,0.304567,0.347234,0.080141
주민센터,-0.017147,0.235233,0.076421,0.129950,0.229134,0.234665,0.368348,0.255012,1.000000,0.195367,0.082806,0.372509,0.416524,0.419179,0.049659
지하철,-0.075061,0.145676,-0.173473,0.021911,0.200857,0.404220,0.190681,0.040482,0.195367,1.000000,0.057867,0.167691,0.179163,0.185607,0.309591


In [19]:
df2.describe()

,신호등_index,교육시설,도서관,버스정류소,복지관,시각장애인협회,안마원,음향신호기,주민센터,지하철,터미널,편의점,약국,병원,관할구별 시각장애인 수
count,24073.000000,24073.000000,24073.000000,24073.000000,24073.000000,24073.000000,24073.000000,24073.000000,24073.000000,24073.000000,24073.000000,24073.000000,24073.000000,24073.000000,24073.000000
mean,12037.000000,7773.743230,-0.028490,0.300843,0.092934,0.152917,0.166696,0.579197,0.180335,0.163150,-0.017896,0.275232,0.248555,0.336483,0.069466
std,6949.420851,3824.003828,0.654658,1.082920,0.779362,0.792046,0.836233,1.078688,0.938423,0.764879,0.672622,1.255208,0.983846,1.031792,0.886934
min,1.000000,89.242346,-1.405923,-0.903424,-1.314548,-1.160292,-1.079072,-0.048704,-1.196184,-0.945239,-1.435128,-0.980739,-0.739859,-0.596495,-1.623377
25%,6019.000000,4897.541539,-0.542415,-0.374800,-0.469279,-0.422594,-0.438610,-0.035042,-0.436971,-0.414970,-0.541514,-0.412593,-0.410826,-0.325807,-0.362013
50%,12037.000000,7265.552455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,18055.000000,10426.432420,0.457585,0.625200,0.530721,0.577406,0.561390,0.964958,0.563029,0.585030,0.458486,0.587407,0.589174,0.674193,0.637987
max,24073.000000,18097.306950,2.057339,18.433515,5.138096,3.167422,4.748266,24.160154,9.157024,4.382342,2.118673,23.106555,14.205717,17.796768,1.824675


In [20]:
#선형회귀
from sklearn.linear_model import LinearRegression
model1 = LinearRegression()
model1.fit(x_train, y_train)
from sklearn.metrics import mean_squared_error
some_predicted1 = model1.predict(x_test)
mse1 = np.sqrt(mean_squared_error(some_predicted1, y_test))
print('평균제곱근오차', mse1)

평균제곱근오차 0.44827759498042624


In [21]:
some_predicted1

array([0.4595141 , 0.55974721, 0.38060129, ..., 0.57932457, 0.40275088,
       0.49071786])

In [22]:
y_test

2799    0
3780    1
6887    1
7640    0
2925    1
       ..
2251    0
8083    0
2033    0
6202    1
8581    0
Name: label, Length: 1811, dtype: int32

In [23]:
#로지스틱
from sklearn.linear_model import LogisticRegression
model2 = LogisticRegression()
model2.fit(x_train, y_train)
print(model2.score(x_train, y_train))

0.7070274748032583


In [48]:
model2.predict_proba(df2)

C:\Users\user\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- 교육시설 
- 지하철 
- 터미널 
- 편의점 
Feature names seen at fit time, yet now missing:
- 교육시설
- 지하철
- 터미널
- 편의점

  warnings.warn(message, FutureWarning)


array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]])

In [49]:
model2.predict_proba(x_test)

array([[0.54941252, 0.45058748],
       [0.35568029, 0.64431971],
       [0.3875684 , 0.6124316 ],
       ...,
       [0.2773559 , 0.7226441 ],
       [0.61795136, 0.38204864],
       [0.65652921, 0.34347079]])

In [24]:
#의사결정트리
from sklearn.tree import DecisionTreeRegressor
model3 = DecisionTreeRegressor()
model3.fit(x_train, y_train)
from sklearn.metrics import mean_squared_error
some_predicted3 = model3.predict(x_test)
mse3 = np.sqrt(mean_squared_error(some_predicted3, y_test))
print('평균제곱근오차', mse3)

평균제곱근오차 0.3917988643204918


In [ ]:
#의사결정트리
from sklearn.tree import DecisionTreeClassifier
model3 = DecisionTreeClassifier()
model3.fit(x_train, y_train)
from sklearn.metrics import mean_squared_error
some_predicted3 = model3.predict(x_test)
mse3 = np.sqrt(mean_squared_error(some_predicted3, y_test))
print('평균제곱근오차', mse3)

In [ ]:
model2.predict_proba(df2)

In [25]:
accuracy_score(y_test, some_predicted3)

0.8464936499171728

In [26]:
#SVM
from sklearn import svm
model4 = svm.SVC()
model4.fit(x_train, y_train)
some_predicted4=model4.predict(x_test)
accuracy_score(y_test, some_predicted4)

0.737161789066814

In [39]:
#랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
model5 = RandomForestClassifier()
model5.fit(x_train, y_train)
from sklearn.metrics import mean_squared_error
some_predicted5 = model5.predict(x_test)
mse5= np.sqrt(mean_squared_error(some_predicted5,y_test))
print('평균제곱근오차', mse5)

평균제곱근오차 0.2606113527744329


In [41]:
result5=model5.predict_proba(df2)

C:\Users\user\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- 교육시설 
- 지하철 
- 터미널 
- 편의점 
Feature names seen at fit time, yet now missing:
- 교육시설
- 지하철
- 터미널
- 편의점

  warnings.warn(message, FutureWarning)


In [44]:
print(result5)
print(pd.DataFrame(result5))

[[0.36 0.64]
 [0.57 0.43]
 [0.55 0.45]
 ...
 [0.58 0.42]
 [0.37 0.63]
 [0.5  0.5 ]]
          0     1
0      0.36  0.64
1      0.57  0.43
2      0.55  0.45
3      0.83  0.17
4      0.72  0.28
...     ...   ...
24068  0.45  0.55
24069  0.45  0.55
24070  0.58  0.42
24071  0.37  0.63
24072  0.50  0.50

[24073 rows x 2 columns]


In [43]:
result5_=model5.predict(df2)
print(result5_)

[1 0 0 ... 0 1 0]


C:\Users\user\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- 교육시설 
- 지하철 
- 터미널 
- 편의점 
Feature names seen at fit time, yet now missing:
- 교육시설
- 지하철
- 터미널
- 편의점

  warnings.warn(message, FutureWarning)


In [40]:
from sklearn.ensemble import RandomForestRegressor
model6 = RandomForestRegressor()
model6.fit(x_train, y_train)
from sklearn.metrics import mean_squared_error
some_predicted6 = model6.predict(x_test)
mse6= np.sqrt(mean_squared_error(some_predicted6,y_test))
print('평균제곱근오차', mse6)

평균제곱근오차 0.2701489455624705


In [28]:
index=df2[['신호등_index']]
df2=df2.drop(['신호등_index'],axis=1)

In [29]:
index['신호등_index']

0            1
1            2
2            3
3            4
4            5
         ...  
24068    24069
24069    24070
24070    24071
24071    24072
24072    24073
Name: 신호등_index, Length: 24073, dtype: int64

result1=pd.DataFrame(model1.predict(df2))
result2=pd.DataFrame(model2.predict(df2))
result3=pd.DataFrame(model3.predict(df2))
result4=pd.DataFrame(model4.predict(df2))
result5=pd.DataFrame(model5.predict(df2))

In [46]:
#result1=model1.predict_proba(df2)
result2=model2.predict_proba(df2)
result3=model3.predict_proba(df2)
result4=model4.predict_proba(df2)
result5=model5.predict_proba(df2)
result6=model5.predict_proba(df2)

C:\Users\user\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- 교육시설 
- 지하철 
- 터미널 
- 편의점 
Feature names seen at fit time, yet now missing:
- 교육시설
- 지하철
- 터미널
- 편의점

  warnings.warn(message, FutureWarning)


AttributeError: 'DecisionTreeRegressor' object has no attribute 'predict_proba'

In [30]:
model1.predict(df2)

C:\Users\user\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- 교육시설 
- 지하철 
- 터미널 
- 편의점 
Feature names seen at fit time, yet now missing:
- 교육시설
- 지하철
- 터미널
- 편의점

  warnings.warn(message, FutureWarning)


array([175.39316229, 151.19438309, 130.32947768, ..., 164.39560151,
       164.39153136, 143.69934135])

In [31]:
x_train

,교육시설,도서관,버스정류소,복지관,시각장애인협회,안마원,음향신호기,주민센터,지하철,터미널,편의점,약국,병원,관할구별 시각장애인 수
2958,-0.231058,-1.204980,0.038820,0.284013,-0.499741,0.400757,0.657312,0.429307,0.499006,0.061263,-0.608267,0.305196,-0.451388,0.645969
1529,-0.330791,-0.975772,-0.286690,0.262853,0.014101,-0.255878,-0.461581,-0.560880,-0.113247,0.020884,-0.010431,-0.513720,-0.401314,0.429194
4259,1.159242,3.151409,0.624342,2.694660,2.677731,4.770392,9.584699,3.525092,4.837739,2.214103,3.143556,5.468717,6.352553,-0.470588
2146,0.011157,-0.104647,0.200895,0.203794,-0.789830,0.495340,1.063755,-0.263037,0.310374,-0.276968,0.566865,0.318230,0.140734,0.000000
7846,-0.434291,0.183662,-0.385814,0.020238,0.660944,-0.363303,0.236292,-0.209653,-0.247023,-0.125547,-0.211920,0.273004,0.032061,-0.014161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3662,0.332208,0.015889,-0.328275,0.593323,-0.768261,-0.182281,-0.117693,-0.345512,0.608095,-0.387039,0.195659,-0.588799,-0.472579,-0.369281
4375,0.790158,2.186888,-0.381063,2.201922,2.244156,1.998320,0.313953,5.263110,3.150893,0.947229,2.866595,7.080762,4.507455,-0.470588
2721,-0.829495,-0.342311,-0.252461,-0.664903,-0.037871,-0.381755,-0.460305,-0.006478,-0.295799,-0.632552,-0.346602,-0.505286,-0.340492,0.006536
6023,-0.651021,-0.592578,0.030840,-0.651903,-0.242191,0.049291,0.024936,-0.343115,-0.197386,-0.446063,-0.059926,-0.470435,-0.339216,-0.208061


In [32]:
x_test

,교육시설,도서관,버스정류소,복지관,시각장애인협회,안마원,음향신호기,주민센터,지하철,터미널,편의점,약국,병원,관할구별 시각장애인 수
2799,-0.391928,-0.695927,-0.050498,-0.350034,-0.588133,-0.213920,-0.312925,0.156704,0.027767,0.106582,0.186107,-0.109310,0.059923,0.645969
3780,0.026084,-0.595797,-0.445278,-0.597852,-1.089271,-0.110027,-0.039334,-0.326292,-0.015427,-0.510716,-0.207291,0.079445,-0.088028,-0.675381
6887,1.434764,0.390096,1.260892,0.308350,0.568671,1.160014,-0.326419,4.392383,2.009345,1.543270,-0.076053,-0.341389,-0.233315,-0.790850
7640,1.305213,0.199607,1.823841,0.189020,0.427343,0.284819,-0.024247,2.745847,1.563557,1.297646,1.302589,1.218585,1.079060,-0.790850
2925,-0.124541,-0.950204,0.284470,0.209316,-0.775210,-0.104547,0.396873,-0.443478,-0.162751,0.015060,-0.142237,-0.367017,-0.273971,0.645969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2251,-0.853191,-0.124789,-0.404844,-0.467632,-0.031462,0.063032,-0.046322,-0.285948,-0.072313,-0.357661,-0.061730,-0.349673,-0.303822,0.006536
8083,-0.688692,-0.046635,0.420811,-0.278658,0.385510,0.028788,0.158797,-0.118883,-0.238646,-0.245034,-0.256790,-0.493390,-0.432605,-0.014161
2033,-0.009069,0.093055,-0.167346,-0.167283,-0.597445,0.421718,-0.303211,0.257661,0.156535,0.003675,-0.314405,-0.366313,-0.444354,0.000000
6202,-0.750188,-0.432128,1.104661,-0.452177,-0.160156,0.020817,-0.410320,-0.293029,-0.192148,-0.561452,1.126499,0.408010,-0.209677,-0.208061


In [33]:
y_train

2958    0
1529    1
4259    0
2146    0
7846    1
       ..
3662    0
4375    0
2721    0
6023    1
2618    1
Name: label, Length: 7243, dtype: int32

In [34]:
#선형회귀
print(result1)

NameError: name 'result1' is not defined

In [ ]:
#로지스틱
pd.set_option('display.max_columns', None)
print(result2)

In [ ]:
#의사결정트리
print(result3)

In [ ]:
#SVM
print(result4)

In [ ]:
#랜덤포레스트
print(result5)

In [ ]:
pd.DataFrame({'Index':df2.index+1, 'y_pred':result3 }).to_csv('선형회귀.csv',index=False)
#check

newfile=pd.read_csv('선형회귀.csv')
newfile

In [89]:
df=pd.read_csv('서울데이터셋_예측.csv', encoding='euckr')

In [90]:
df

,신호등_index,관리번호,관할구,주소,X좌표,Y좌표,교육시설,도서관,버스정류소,복지관,...,주민센터,지하철,터미널,편의점,약국,병원,교통사고,관할구별 시각장애인 수,label,smart_label
0,1,03-0000013905,강남구,강남구 대치동 973-12 대,205374.9878,544360.3235,8828.504490,1272.720157,63.539989,1346.410759,...,284.085577,506.050553,4951.938755,38.526782,41.464028,37.449544,704.479577,1551,1,0.697482
1,2,03-0000064343,강남구,강남구 역삼동 712 공,203974.3625,544362.8187,7609.817483,2564.026031,56.791601,889.563384,...,346.584163,647.881226,3578.020909,164.459202,109.535357,84.957371,1034.071646,1551,1,0.652321
2,3,03-0000065097,강남구,강남구 역삼동 633-16 대,202722.1824,544404.9315,6557.443582,3724.820316,88.930535,605.690941,...,469.181845,212.613690,2362.775142,168.592329,26.863093,26.863093,461.051083,1551,1,0.680306
3,4,03-0000014413,강남구,강남구 삼성동 186-6천,205839.8146,546182.1334,8603.064701,2586.538263,110.674797,2188.520623,...,325.610619,447.503186,3299.653565,93.347071,263.335148,247.677596,297.863153,1551,0,0.354870
4,5,03-0000066954,강남구,강남구 일원동 306유,207106.7500,544477.2562,10352.677580,1048.237676,396.732442,921.576718,...,792.651712,623.070900,4058.568839,442.528962,541.561810,599.914988,1526.681283,1551,0,0.162643
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24068,24069,03-0000083555,중랑구,중랑구 신내동 660대,208276.5375,556330.4937,7230.450246,2681.460032,35.737890,298.156171,...,47.062448,699.313006,1160.385186,84.847481,33.221670,57.142015,1167.949241,2094,1,0.617122
24069,24070,03-0000083554,중랑구,중랑구 신내동 660대,208276.5375,556330.4937,7230.450246,2681.460032,35.737890,298.156171,...,47.062448,699.313006,1160.385186,84.847481,33.221670,57.142015,1167.949241,2094,1,0.617122
24070,24071,03-0000001936,중랑구,중랑구 면목동 534-32도,208475.4678,554282.8431,8283.776312,4607.601742,0.736851,1820.796008,...,514.306317,714.002440,956.527579,170.793254,154.857993,106.395229,1087.311174,2094,0,0.344868
24071,24072,03-0000083287,중랑구,중랑구 면목동 454-35대,208556.5389,554420.2290,8282.427777,4510.148534,25.926075,1685.624452,...,357.276709,801.994989,865.386136,63.388223,9.139335,25.484132,930.583168,2094,0,0.486899


In [91]:
df['교통사고'] = np.where(df['교통사고'] > 200, 0, df['교통사고'] )

In [92]:
df

,신호등_index,관리번호,관할구,주소,X좌표,Y좌표,교육시설,도서관,버스정류소,복지관,...,주민센터,지하철,터미널,편의점,약국,병원,교통사고,관할구별 시각장애인 수,label,smart_label
0,1,03-0000013905,강남구,강남구 대치동 973-12 대,205374.9878,544360.3235,8828.504490,1272.720157,63.539989,1346.410759,...,284.085577,506.050553,4951.938755,38.526782,41.464028,37.449544,0.0,1551,1,0.697482
1,2,03-0000064343,강남구,강남구 역삼동 712 공,203974.3625,544362.8187,7609.817483,2564.026031,56.791601,889.563384,...,346.584163,647.881226,3578.020909,164.459202,109.535357,84.957371,0.0,1551,1,0.652321
2,3,03-0000065097,강남구,강남구 역삼동 633-16 대,202722.1824,544404.9315,6557.443582,3724.820316,88.930535,605.690941,...,469.181845,212.613690,2362.775142,168.592329,26.863093,26.863093,0.0,1551,1,0.680306
3,4,03-0000014413,강남구,강남구 삼성동 186-6천,205839.8146,546182.1334,8603.064701,2586.538263,110.674797,2188.520623,...,325.610619,447.503186,3299.653565,93.347071,263.335148,247.677596,0.0,1551,0,0.354870
4,5,03-0000066954,강남구,강남구 일원동 306유,207106.7500,544477.2562,10352.677580,1048.237676,396.732442,921.576718,...,792.651712,623.070900,4058.568839,442.528962,541.561810,599.914988,0.0,1551,0,0.162643
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24068,24069,03-0000083555,중랑구,중랑구 신내동 660대,208276.5375,556330.4937,7230.450246,2681.460032,35.737890,298.156171,...,47.062448,699.313006,1160.385186,84.847481,33.221670,57.142015,0.0,2094,1,0.617122
24069,24070,03-0000083554,중랑구,중랑구 신내동 660대,208276.5375,556330.4937,7230.450246,2681.460032,35.737890,298.156171,...,47.062448,699.313006,1160.385186,84.847481,33.221670,57.142015,0.0,2094,1,0.617122
24070,24071,03-0000001936,중랑구,중랑구 면목동 534-32도,208475.4678,554282.8431,8283.776312,4607.601742,0.736851,1820.796008,...,514.306317,714.002440,956.527579,170.793254,154.857993,106.395229,0.0,2094,0,0.344868
24071,24072,03-0000083287,중랑구,중랑구 면목동 454-35대,208556.5389,554420.2290,8282.427777,4510.148534,25.926075,1685.624452,...,357.276709,801.994989,865.386136,63.388223,9.139335,25.484132,0.0,2094,0,0.486899


In [93]:
df['교통사고'].value_counts()

0.000000      23194
66.640986         3
79.667675         2
132.387116        2
59.339833         2
              ...  
112.588300        1
132.334268        1
146.663765        1
165.033365        1
144.394625        1
Name: 교통사고, Length: 819, dtype: int64

In [94]:
df['교통사고'] = np.where(df['교통사고'] != 200, 200-df['교통사고'] , df['교통사고'] )

In [95]:
df['교통사고'].value_counts()

200.000000    23194
133.359014        3
120.332325        2
67.612884         2
140.660167        2
              ...  
87.411700         1
67.665732         1
53.336235         1
34.966635         1
55.605375         1
Name: 교통사고, Length: 819, dtype: int64

In [96]:
from sklearn.preprocessing import MinMaxScaler

s = MinMaxScaler()
df['교통사고']=s.fit_transform(df[['교통사고']])

In [97]:
df['교통사고'].value_counts()

1.000000    23194
0.666165        3
0.600908        2
0.336812        2
0.702739        2
            ...  
0.435993        1
0.337077        1
0.265294        1
0.173272        1
0.276661        1
Name: 교통사고, Length: 819, dtype: int64

In [98]:
df['교통사고'] = np.where(df['교통사고'] == 1, 0 , df['교통사고'] )

In [99]:
df['교통사고'].value_counts()

0.000000    23195
0.666165        3
0.600908        2
0.702739        2
0.336812        2
            ...  
0.435993        1
0.337077        1
0.265294        1
0.173272        1
0.276661        1
Name: 교통사고, Length: 818, dtype: int64

In [100]:
df.columns

Index(['신호등_index', '관리번호', '관할구', '주소', 'X좌표', 'Y좌표', '교육시설 ', '도서관', '버스정류소',
       '복지관', '시각장애인협회', '안마원', '음향신호기', '주민센터', '지하철', '터미널 ', '편의점 ', '약국',
       '병원', '교통사고', '관할구별 시각장애인 수', 'label', 'smart_label'],
      dtype='object')

In [101]:
dfs=df[['신호등_index','X좌표', 'Y좌표','교통사고','smart_label']]
dfs

,신호등_index,X좌표,Y좌표,교통사고,smart_label
0,1,205374.9878,544360.3235,0.0,0.697482
1,2,203974.3625,544362.8187,0.0,0.652321
2,3,202722.1824,544404.9315,0.0,0.680306
3,4,205839.8146,546182.1334,0.0,0.354870
4,5,207106.7500,544477.2562,0.0,0.162643
...,...,...,...,...,...
24068,24069,208276.5375,556330.4937,0.0,0.617122
24069,24070,208276.5375,556330.4937,0.0,0.617122
24070,24071,208475.4678,554282.8431,0.0,0.344868
24071,24072,208556.5389,554420.2290,0.0,0.486899


In [102]:
dfs['smart_label2']=dfs['smart_label']*100

C:\Users\user\AppData\Local\Temp\ipykernel_11356\95005019.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs['smart_label2']=dfs['smart_label']*100


In [103]:
dfs['교통사고2']=dfs['교통사고']*100

C:\Users\user\AppData\Local\Temp\ipykernel_11356\3761175378.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs['교통사고2']=dfs['교통사고']*100


In [104]:
dfs['result']=(dfs['smart_label2']*0.9+dfs['교통사고2']*0.1)
dfs

C:\Users\user\AppData\Local\Temp\ipykernel_11356\611327621.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs['result']=dfs['smart_label2']*0.9+dfs['교통사고2']*0.1


,신호등_index,X좌표,Y좌표,교통사고,smart_label,smart_label2,교통사고2,result
0,1,205374.9878,544360.3235,0.0,0.697482,69.748225,0.0,62.773402
1,2,203974.3625,544362.8187,0.0,0.652321,65.232070,0.0,58.708863
2,3,202722.1824,544404.9315,0.0,0.680306,68.030572,0.0,61.227515
3,4,205839.8146,546182.1334,0.0,0.354870,35.486989,0.0,31.938290
4,5,207106.7500,544477.2562,0.0,0.162643,16.264267,0.0,14.637841
...,...,...,...,...,...,...,...,...
24068,24069,208276.5375,556330.4937,0.0,0.617122,61.712230,0.0,55.541007
24069,24070,208276.5375,556330.4937,0.0,0.617122,61.712230,0.0,55.541007
24070,24071,208475.4678,554282.8431,0.0,0.344868,34.486835,0.0,31.038151
24071,24072,208556.5389,554420.2290,0.0,0.486899,48.689946,0.0,43.820951


In [67]:
dfs['교통사고2'].value_counts()

0.0    23194
0.9      879
Name: 교통사고2, dtype: int64

In [65]:
dfs['smart_label2'].value_counts()

0.006159    6
0.004237    6
0.011994    5
0.895290    4
0.014863    4
           ..
0.391482    1
0.491012    1
0.415642    1
0.486789    1
0.322750    1
Name: smart_label2, Length: 22416, dtype: int64

In [107]:
dfs.to_csv('결과.csv',index=False,encoding='euckr')
#check